In [2]:
import os
import netCDF4 as nc4
import numpy as np
import toolbox.collection as collection
import toolbox.IcesheetCHANGES as changes

### Print available collections

In [3]:
print(collection.print_collections())

Available collections: 

MEaSUREs Annual Antarctic Ice Velocity Maps V001
Short name: MEaSUREs Antarctic Annual Velocity

MEaSUREs Greenland Quarterly Ice Sheet Velocity Mosaics from SAR and Landsat V005
Short name: MEaSUREs Greenland Quarterly Velocity

MEaSUREs Greenland Monthly Ice Sheet Velocity Mosaics from SAR and Landsat, Version 5
Short name: MEaSUREs Greenland Monthly Velocity

ATLAS/ICESat-2 L3B Gridded Antarctic and Arctic Land Ice Height, Version 2
Short name: ATL14 Antarctic Elevation

ATLAS/ICESat-2 L3B Gridded Antarctic and Arctic Land Ice Height Change, Version 2
Short name: ATL15 Antarctic Elevation

None


### Set Options

In [4]:
collection_key = 'MEaSUREs Antarctic Annual Velocity'
region_name = 'Antarctic'       # 'Antarctica' or 'Greenland'
data_type = 'velocity'          # 'velocity' or 'elevation'
#glacier_name = 'Pine Island'     
glacier_name = 'Antarctica'

project_folder = '/Users/tara/Documents/SJSU/MLML/Projects/CHANGES/Examples'
#data_folder = '/Users/tara/Documents/SJSU/MLML/Test Data/ncs/'
              #'/Volumes/Seagate/CHANGES/data_repository/tutorial'
data_folder = '/Volumes/Seagate/CHANGES/data_repository/tutorial'

### Initialize Changes Object

In [5]:
collection_id = collection.collection(collection_key)
print('Collection ID: ', collection_id)

IC = changes.AntarcticCHANGES(project_folder, data_folder, collection_key, collection_id, region_name, data_type)
IC.velocity_grid_posting = 50000
IC.glacier_name = glacier_name
IC.print_attributes()

Collection ID:  C2245171699-NSIDC_ECS
Region name:		 Antarctic
Collection name:	 MEaSUREs Annual Antarctic Ice Velocity Maps V001
Collection short name:	 MEaSUREs Antarctic Annual Velocity
Collection ID: 		 C2245171699-NSIDC_ECS
Data type:		 Velocity
Projection:		 3031
Download path:		 /Volumes/Seagate/CHANGES/data_repository/tutorial/Antarctic/Velocity/MEaSUREs Antarctic Annual Velocity/Data
Metadata path:		 /Users/tara/Documents/SJSU/MLML/Projects/CHANGES/Examples/Antarctic/Velocity/Metadata


In [6]:
# Estimated extents of pine island glacier
min_x = -1670811.49223
min_y = -342750.13133
max_x = -1546045.93205
max_y = -279597.95439
IC.extents = [min_x, min_y, max_x, max_y]  #minX, minY, maxX, maxY

In [7]:
# Extents for Antarctica (estimated) TEMPORARY
x = [-2525725.161631, -1969164.736031, -529013.605469, -314793.229530, -85039.233025, -195124.382447, 212361.833364, 323206.286926, 1203859.622106, 2442332.087405, 2477479.184929, 465453.711660, 2158704.981460, -736133.500297]
y = [1621194.950833, -329249.148704, -1299650.368502, -1124088.890017, -766824.476681, -505840.940327, -1270088.123800, -1970165.590919, -2109988.900373, -1020436.845258, 313287.294861, -2072759.504125, 1580566.125097, 1058972.421230]

min_x = min(x)
max_x = max(x)
min_y = min(y)
max_y = max(y)
IC.extents = [min_x, min_y, max_x, max_y]  #minX, minY, maxX, maxY

### Get files to stack

In [19]:
files = os.listdir(IC.download_path)
files = [f for f in files if f.endswith('.nc')]

# Check the file names are as expected
print(files)

['Antarctica_ice_velocity_2000_2001_1km_v01.1.nc', 'Antarctica_ice_velocity_2005_2006_1km_v01.nc', 'Antarctica_ice_velocity_2006_2007_1km_v01.nc', 'Antarctica_ice_velocity_2007_2008_1km_v01.nc', 'Antarctica_ice_velocity_2008_2009_1km_v01.nc', 'Antarctica_ice_velocity_2009_2010_1km_v01.nc', 'Antarctica_ice_velocity_2010_2011_1km_v01.nc', 'Antarctica_ice_velocity_2011_2012_1km_v01.nc', 'Antarctica_ice_velocity_2012_2013_1km_v01.nc', 'Antarctica_ice_velocity_2013_2014_1km_v01.nc', 'Antarctica_ice_velocity_2014_2015_1km_v01.nc', 'Antarctica_ice_velocity_2015_2016_1km_v01.nc', 'Antarctica_ice_velocity_2016_2017_1km_v01.nc', 'Antarctica_ice_velocity_2017_2018_1km_v01.1.nc', 'Antarctica_ice_velocity_2018_2019_1km_v01.1.nc', 'Antarctica_ice_velocity_2019_2020_1km_v01.1.nc']


### Set extents

ChatGPT:
Based on my knowledge cutoff in September 2021, the Pine Island Glacier is approximately located between approximately 74.5°S to 75.5°S latitude and 99.5°W to 102.5°W longitude in West Antarctica. However, please note that these coordinates are approximate, and for the most accurate and up-to-date information, it’s recommended to refer to recent scientific sources or maps.

espg.io transform:

min_x = -1670811.49223

min_y = -342750.13133

max_x = -1546045.93205

max_y = -279597.95439

In [20]:
x = [-2525725.161631, -1969164.736031, -529013.605469, -314793.229530, -85039.233025, -195124.382447, 212361.833364, 323206.286926, 1203859.622106, 2442332.087405, 2477479.184929, 465453.711660, 2158704.981460, -736133.500297]
y = [1621194.950833, -329249.148704, -1299650.368502, -1124088.890017, -766824.476681, -505840.940327, -1270088.123800, -1970165.590919, -2109988.900373, -1020436.845258, 313287.294861, -2072759.504125, 1580566.125097, 1058972.421230]

print(min(x), max(x))
print(min(y), max(y))

-2525725.161631 2477479.184929
-2109988.900373 1621194.950833


In [21]:
def create_grid_from_extents(extents,posting,epsg):

    # TODO get extents of Antarctica from bedmachine in epsg:3031
    # temporary: points are a collection of random boundary points of Antarctica in the epsg:3031 projection 
    x = [-2525725.161631, -1969164.736031, -529013.605469, -314793.229530, -85039.233025, -195124.382447, 212361.833364, 323206.286926, 1203859.622106, 2442332.087405, 2477479.184929, 465453.711660, 2158704.981460, -736133.500297]
    y = [1621194.950833, -329249.148704, -1299650.368502, -1124088.890017, -766824.476681, -505840.940327, -1270088.123800, -1970165.590919, -2109988.900373, -1020436.845258, 313287.294861, -2072759.504125, 1580566.125097, 1058972.421230]
    
    minBedMachineX = min(x)
    maxBedMachineX = max(x)
    minBedMachineY = min(y)
    maxBedMachineY = max(y)

    x = np.arange(minBedMachineX, maxBedMachineX, posting)
    y = np.arange(minBedMachineY, maxBedMachineY, posting)

    x = x[x >= extents[0]]
    x = x[x <= extents[2]]

    y = y[y >= extents[1]]
    y = y[y <= extents[3]]
    
    return(x,y)


def create_velocity_grids_from_extents(GD_object):
    velocity_grid_x, velocity_grid_y = \
        create_grid_from_extents(GD_object.extents, GD_object.velocity_grid_posting, GD_object.velocity_grid_epsg)

    GD_object.velocity_grid_x = velocity_grid_x
    GD_object.velocity_grid_y = velocity_grid_y

def create_elevation_grids_from_extents(GD_object):
    elevation_grid_x, elevation_grid_y = \
        create_grid_from_extents(GD_object.extents, GD_object.elevation_grid_posting, GD_object.elevation_grid_epsg)

    GD_object.elevation_grid_y = elevation_grid_y
    GD_object.elevation_grid_x = elevation_grid_x

In [22]:
create_velocity_grids_from_extents(IC)

In [23]:
def get_arrs(IC, file):

    ds = nc4.Dataset(os.path.join(IC.download_path, file), 'r')

    vx = ds.variables['VX'][:]
    vy = ds.variables['VY'][:]
    ex = ds.variables['ERRX'][:]
    ey = ds.variables['ERRY'][:]
    x = ds.variables['x'][:]
    y = ds.variables['y'][:]

    # TODO: Should array be initialized to the nan value -99999
    vx_array = np.zeros((len(IC.velocity_grid_y), len(IC.velocity_grid_x)))
    vy_array = np.zeros((len(IC.velocity_grid_y), len(IC.velocity_grid_x)))
    vv_array = np.zeros((len(IC.velocity_grid_y), len(IC.velocity_grid_x)))
    errx_array = np.zeros((len(IC.velocity_grid_y), len(IC.velocity_grid_x)))
    erry_array = np.zeros((len(IC.velocity_grid_y), len(IC.velocity_grid_x)))

    for j in range(len(IC.velocity_grid_y)):
        y_index = np.argmin(np.abs(y - IC.velocity_grid_y[j]))
        for i in range(len(IC.velocity_grid_x)):
            x_index = np.argmin(np.abs(x - IC.velocity_grid_x[i]))
            vx_array[j, i] = vx[y_index, x_index]
            vy_array[j, i] = vy[y_index, x_index]
            vv_array[j, i] = np.sqrt(vx_array[j, i]**2 + vy_array[j, i]**2)
            errx_array[j, i] = ex[y_index, x_index]
            erry_array[j, i] = ey[y_index, x_index]
    
    ds.close()
    return vx_array, vy_array, vv_array, errx_array, erry_array

In [24]:
vx_grids = []
vy_grids = []
errx_grids = []
erry_grids = []
vv_grids = []
dates_start = []
dates_end = []
file_names = []

for file in files:
    file_name = file.split('.')[0]
    file_names.append(file_name)
    print(file_name)

    start = file_name.split('_')[3]
    end = file_name.split('_')[4]
    dates_start.append(start)
    dates_end.append(end)
    print(start, end)

    vx_array, vy_array, vv_array, errx_array, erry_array = get_arrs(IC, file)
    vx_grids.append(vx_array)
    vy_grids.append(vy_array)
    errx_grids.append(errx_array)
    erry_grids.append(erry_array)
    vv_grids.append(vv_array)


Antarctica_ice_velocity_2000_2001_1km_v01
2000 2001


/var/folders/5j/sp7zw9k97sdfqkf7m2vn2m4h0000gn/T/ipykernel_10700/1857528055.py:25: UserWarning: Warning: converting a masked element to nan.
  errx_array[j, i] = ex[y_index, x_index]
/var/folders/5j/sp7zw9k97sdfqkf7m2vn2m4h0000gn/T/ipykernel_10700/1857528055.py:26: UserWarning: Warning: converting a masked element to nan.
  erry_array[j, i] = ey[y_index, x_index]


Antarctica_ice_velocity_2005_2006_1km_v01
2005 2006
Antarctica_ice_velocity_2006_2007_1km_v01
2006 2007
Antarctica_ice_velocity_2007_2008_1km_v01
2007 2008
Antarctica_ice_velocity_2008_2009_1km_v01
2008 2009
Antarctica_ice_velocity_2009_2010_1km_v01
2009 2010
Antarctica_ice_velocity_2010_2011_1km_v01
2010 2011
Antarctica_ice_velocity_2011_2012_1km_v01
2011 2012
Antarctica_ice_velocity_2012_2013_1km_v01
2012 2013
Antarctica_ice_velocity_2013_2014_1km_v01
2013 2014
Antarctica_ice_velocity_2014_2015_1km_v01
2014 2015
Antarctica_ice_velocity_2015_2016_1km_v01
2015 2016
Antarctica_ice_velocity_2016_2017_1km_v01
2016 2017


/var/folders/5j/sp7zw9k97sdfqkf7m2vn2m4h0000gn/T/ipykernel_10700/1857528055.py:22: UserWarning: Warning: converting a masked element to nan.
  vx_array[j, i] = vx[y_index, x_index]
/var/folders/5j/sp7zw9k97sdfqkf7m2vn2m4h0000gn/T/ipykernel_10700/1857528055.py:23: UserWarning: Warning: converting a masked element to nan.
  vy_array[j, i] = vy[y_index, x_index]


Antarctica_ice_velocity_2017_2018_1km_v01
2017 2018
Antarctica_ice_velocity_2018_2019_1km_v01
2018 2019
Antarctica_ice_velocity_2019_2020_1km_v01
2019 2020


In [25]:
def output_data_stack(GD_object, vx_grids, vy_grids, vv_grids, errx_grids, erry_grids, dates_start, dates_end):

    if not os.path.exists(os.path.join(IC.download_path, 'output_files')):
        os.makedirs(os.path.join(IC.download_path, 'output_files'))
        
    output_file = os.path.join(IC.download_path, 'output_files', IC.short_name + "_" + IC.glacier_name + "_stack.nc")

    data = nc4.Dataset(output_file, "w", format="NETCDF4")

    data.createDimension('y',len(GD_object.velocity_grid_y))
    data.createDimension('x',len(GD_object.velocity_grid_x))
    xvar = data.createVariable('x','f4',("x",))
    yvar = data.createVariable('y','f4',("y",))

    xvar[:]=GD_object.velocity_grid_x
    yvar[:]=GD_object.velocity_grid_y

    for dd in range(len(dates_start)):
        grp = data.createGroup(dates_start[dd] + '-' + dates_end[dd])
        vx = grp.createVariable('VX','f4',("y","x"))
        vy = grp.createVariable('VY','f4',("y","x"))
        vv = grp.createVariable('VV','f4',("y","x"))

        errx = grp.createVariable('ERRX','f4',("y","x"))
        erry = grp.createVariable('ERRY','f4',("y","x"))

        vx[:,:] = vx_grids[dd]
        vy[:,:] = vy_grids[dd]
        vv[:,:] = vv_grids[dd]

        errx[:,:] = errx_grids[dd]
        erry[:,:] = erry_grids[dd]

    data.close()

    message = '        Saved file to ' + output_file
    GD_object.output_summary += '\n' + message
    print(message)
    

In [26]:
output_data_stack(IC, vx_grids, vy_grids, vv_grids, errx_grids, erry_grids, dates_start, dates_end)

        Saved file to /Volumes/Seagate/CHANGES/data_repository/tutorial/Antarctic/Velocity/MEaSUREs Antarctic Annual Velocity/Data/output_files/MEaSUREs Antarctic Annual Velocity_Antarctica_stack.nc
